# Scraping Websites

In [1]:
#Python Imports
import os
import sys
import csv
import json
import time
import itertools
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from selenium import webdriver
from IPython.display import Image
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC


#Selenium Imports
chrome_options = Options()
#chrome_options.add_argument("--headless") # Ensure GUI is off
#chrome_options.add_argument("--no-sandbox")
browser = webdriver.Chrome(options=chrome_options)
browser.implicitly_wait(15) # seconds

* First pags of DuckDuckGo (can scrape more)
* First page Mojeek, not always results for all zip codes
* First page Yahoo
* First page of Bing
* First page of Yellow Pages
* First page of Brave Search
* First page of StartPage
* 2 pages of Google

# Set Parameters & Scraping

In [2]:
#######
search_term_raw = 'le specs'
brand_url = 'https://lespecs.com/pages/stockists'
########

search_term = search_term_raw + " in "
pages = 2
zipcodes = pd.read_json('OnePager/top_500_zipcodes.json')['zip'].apply(lambda x: '{:0>5}'.format(x)).to_list()

rep_zips = [
    '94110', '90210', '10001', '20001', '98101',
    '60601', '77002', '30303', '02108', '33131',
    '80202', '92101', '85004', '98104', '75201',
    '60611', '75205', '19104', '30363', '98109'
]

zipcodesSample = rep_zips#['94123', '19104', '77494']

## DuckDuckGo

In [3]:
duckduckGo = []

#Get Starting Webpage
url = "https://duckduckgo.com/?q=ulla+johnson&va=u&t=he&ia=web"
browser.get(url)

for zipcode in zipcodesSample:
    try:
        #Submit & Search
        search_form = browser.find_element(By.CLASS_NAME, 'js-search-input')
        search_form.clear()
        search_form.send_keys(search_term+zipcode)

        submit = browser.find_element(By.CLASS_NAME, 'search__button')
        #submit = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'search__button')))
        #browser.execute_script("arguments[0].scrollIntoView();", submit)
        time.sleep(1)
        submit.click()
        time.sleep(1)



        #Load/Click Load More Results
        #more_results_button = browser.find_element(By.CLASS_NAME, 'result--more')    

        #UNCOMMENT TO GET MORE RESULTS BUTTON & CLICK
        #more_results_button = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, 'result--more')))
        #more_results_button.click()

        #Collect Results
        results_box = browser.find_element(By.ID, 'links')
        results = results_box.find_elements(By.CLASS_NAME, 'nrn-react-div')


        #Scraping Through the Results
        for idx, store in enumerate(results, start=1):
            resultsInfo = {}
            resultsInfo['Title'] = store.find_element(By.CLASS_NAME, 'ikg2IXiCD14iVX7AdZo1').get_attribute("textContent")
            resultsInfo['RankNumber'] = idx
            resultsInfo['ZipCode'] = zipcode
            resultsInfo['searchTerm'] = search_term+zipcode

            duckduckGo.append(resultsInfo)
        print(zipcode, ' - ', len(results))
    except Exception as e:
        print('ERROR', str(e)[0:75]+"...")
    
    browser.get(url)


94110  -  10
90210  -  10
10001  -  10
20001  -  10
98101  -  10
60601  -  10
77002  -  10
30303  -  10
02108  -  10
33131  -  10
80202  -  10
92101  -  10
85004  -  10
98104  -  10
75201  -  10
60611  -  10
75205  -  10
19104  -  10
30363  -  10
98109  -  10


In [4]:
pd.DataFrame(duckduckGo).head()#.to_csv("pbrDoubles.csv")

,Title,RankNumber,ZipCode,searchTerm
0,Store Locator - Le Specs,1,94110,le specs in 94110
1,Le Specs,2,94110,le specs in 94110
2,Le Specs | Designer | NET-A-PORTER,3,94110,le specs in 94110
3,Shop Le Specs Online | Nordstrom,4,94110,le specs in 94110
4,Shop Le Specs Online | Shopbop,5,94110,le specs in 94110


## StartPage

In [5]:
startPage = []

#Get Starting Webpage
url = 'https://www.startpage.com/en/'
browser.get(url)

for zipcode in zipcodesSample:
    #Submit & Search
    browser.get(url)
    submit=browser.find_element(By.ID, 'search-btn')
    search_form = browser.find_element(By.ID, 'q')
    search_form.clear()
    search_form.send_keys(search_term+zipcode)
    submit.click()
    time.sleep(2)
    
    #Collect Results
    results_box = browser.find_element(By.CLASS_NAME, 'w-gl')
    results = results_box.find_elements(By.CLASS_NAME, 'w-gl__result')

    #Scraping Through the Results
    for idx, store in enumerate(results, start=1):
        resultsInfo = {}
        resultsInfo['Title'] = store.find_element(By.TAG_NAME, 'h3').text
        resultsInfo['RankNumber'] = idx
        resultsInfo['ZipCode'] = zipcode
        resultsInfo['searchTerm'] = search_term+zipcode
        resultsInfo['link'] = store.find_element(By.CLASS_NAME, 'result-link').text
        startPage.append(resultsInfo)
        
    print(zipcode, ' - ', len(results))

94110  -  10
90210  -  10
10001  -  10
20001  -  8
98101  -  10
60601  -  8
77002  -  9
30303  -  10
02108  -  8
33131  -  8
80202  -  8
92101  -  10
85004  -  10
98104  -  8
75201  -  10
60611  -  9
75205  -  8
19104  -  8
30363  -  8
98109  -  8


In [6]:
pd.DataFrame(startPage).head()#.to_csv("pbrDoubles.csv")

,Title,RankNumber,ZipCode,searchTerm,link
0,Stockists - Le Specs,1,94110,le specs in 94110,https://lespecs.com/pages/stockists
1,Le Specs,2,94110,le specs in 94110,https://lespecs.com/
2,HANKERING | BLACK - Le Specs,3,94110,le specs in 94110,https://lespecs.com/products/hankering-black-l...
3,Outta Love | Black Sunglasses - Le Specs,4,94110,le specs in 94110,https://lespecs.com/products/outta-love-black-...
4,FANPLASTICO | ORCHID - Le Specs,5,94110,le specs in 94110,https://lespecs.com/products/fanplastico-orchi...


## Brave Search

In [17]:
braveSearch = []

#Get Starting Webpage
url = 'https://search.brave.com/'
browser.get(url)

for zipcode in zipcodesSample:
    #Submit & Search
    browser.get(url)
    submit=browser.find_element(By.ID, 'submit-button')
    search_form = browser.find_element(By.ID, 'searchbox')
    search_form.clear()
    search_form.send_keys(search_term+zipcode)
    submit.click()
    time.sleep(2)
    
    #Collect Results
    results_box = browser.find_element(By.ID, 'results')
    results = results_box.find_elements(By.CLASS_NAME, 'fdb')

    #Scraping Through the Results
    for idx, store in enumerate(results, start=1):
        resultsInfo = {}
        resultsInfo['Title'] = store.find_element(By.CLASS_NAME, 'snippet-title').get_attribute("textContent").strip()
        resultsInfo['RankNumber'] = idx
        resultsInfo['ZipCode'] = zipcode
        resultsInfo['searchTerm'] = search_term+zipcode
        
        braveSearch.append(resultsInfo)
        
    print(zipcode, ' - ', len(results))
    
    browser.quit()
    browser = webdriver.Chrome(options=chrome_options)
    #browser.get("https://www.espn.com/")
    time.sleep(3)

94110  -  18


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="results"]"}
  (Session info: chrome=112.0.5615.49)
Stacktrace:
0   chromedriver                        0x000000010549b428 chromedriver + 4899880
1   chromedriver                        0x0000000105418a23 chromedriver + 4364835
2   chromedriver                        0x0000000105062bf6 chromedriver + 474102
3   chromedriver                        0x00000001050a64f0 chromedriver + 750832
4   chromedriver                        0x00000001050a6751 chromedriver + 751441
5   chromedriver                        0x00000001050ea834 chromedriver + 1030196
6   chromedriver                        0x00000001050cc58d chromedriver + 906637
7   chromedriver                        0x00000001050e7b5b chromedriver + 1018715
8   chromedriver                        0x00000001050cc333 chromedriver + 906035
9   chromedriver                        0x000000010509655f chromedriver + 685407
10  c

In [20]:
pd.DataFrame(braveSearch)#.sample(5)

,Title,RankNumber,ZipCode,searchTerm
0,Store Locator – Le Specs,1,94110,le specs in 94110
1,Le Specs,2,94110,le specs in 94110
2,Le Specs | Designer | NET-A-PORTER,3,94110,le specs in 94110
3,Amazon.com: Le Specs,4,94110,le specs in 94110
4,Shop Le Specs Online | Nordstrom,5,94110,le specs in 94110
5,Le Specs Sunglasses Review - Must Read This Be...,6,94110,le specs in 94110
6,Shop Le Specs Online | Shopbop,7,94110,le specs in 94110
7,Home – Le Specs,8,94110,le specs in 94110
8,Our Story – Le Specs,9,94110,le specs in 94110
9,Le Specs | Selfridges,10,94110,le specs in 94110


## Yellow Pages

In [21]:
yellowPages = []

#Get Starting Webpage
url = 'https://www.yellowpages.com/'
browser.get(url)

for zipcode in zipcodesSample:
    #Submit & Search
    submit=browser.find_element(By.TAG_NAME, 'button')
    search_form = browser.find_element(By.ID, 'query')
    location_form = browser.find_element(By.ID, 'location')
    search_form.clear()
    search_form.send_keys(search_term_raw)
    location_form.clear()
    location_form.send_keys(zipcode)
    submit.click()
    time.sleep(2)
    
    try:
        #Collect Results
        results_box = browser.find_element(By.CLASS_NAME, 'organic')
        results = results_box.find_elements(By.CLASS_NAME, 'result')

        #Scraping Through the Results
        for idx, store in enumerate(results, start=1):
            resultsInfo = {}
            resultsInfo['Title'] = store.find_element(By.CLASS_NAME, 'business-name').get_attribute("textContent")
            resultsInfo['RankNumber'] = idx
            resultsInfo['ZipCode'] = zipcode
            resultsInfo['searchTerm'] = search_term_raw+zipcode
            yellowPages.append(resultsInfo)
        print(zipcode, ' - ', len(results))
    except Exception as e:
        print('ERROR', str(e)[0:75]+"...")
    
    browser.get(url)

94110  -  30
90210  -  30
10001  -  30
20001  -  30
98101  -  30
60601  -  30
77002  -  30
30303  -  30
02108  -  30
33131  -  30
80202  -  30
92101  -  28
85004  -  30
98104  -  30
75201  -  30
60611  -  30
75205  -  30
19104  -  30
30363  -  30
98109  -  30


In [22]:
pd.DataFrame(yellowPages)#.to_csv("pbrDoubles.csv")

,Title,RankNumber,ZipCode,searchTerm
0,Specs in The City & Cheaters Too,1,94110,le specs94110
1,Spec Ceramics,2,94110,le specs94110
2,Classic Specs Shop - Hayes Valley,3,94110,le specs94110
3,Topflight Specs,4,94110,le specs94110
4,Specs' Twelve Adler Museum Cafe,5,94110,le specs94110
...,...,...,...,...
593,Top Spec Auto Care & Sales,26,98109,le specs98109
594,AmeriSpec,27,98109,le specs98109
595,Osborne Spec Fe,28,98109,le specs98109
596,Hoover Investigation Spec,29,98109,le specs98109


## Mojeek

In [23]:
mojeek = []

#Get Starting Webpage
url = 'https://www.mojeek.com/'
browser.get(url)

for zipcode in zipcodesSample:
    #Submit & Search
    browser.get(url)
    submit=browser.find_element(By.CLASS_NAME, 'search')
    search_form = browser.find_element(By.CLASS_NAME, 'js-search-input')
    search_form.clear()
    search_form.send_keys(search_term+zipcode)
    submit.click()
    time.sleep(2)
    
    try:
        #Collect Results
        results_box = browser.find_element(By.CLASS_NAME, 'results-standard')
        results = results_box.find_elements(By.TAG_NAME, 'li')
    
        #Scraping Through the Results
        for idx, store in enumerate(results, start=1):
            resultsInfo = {}
            resultsInfo['Title'] = store.find_element(By.CLASS_NAME, 'title').text
            resultsInfo['RankNumber'] = idx
            resultsInfo['ZipCode'] = zipcode
            resultsInfo['searchTerm'] = search_term+zipcode

            mojeek.append(resultsInfo)
    except:
        print("no results/error")
        results = []
        
        
    print(zipcode, ' - ', len(results))

94110  -  10
90210  -  10
10001  -  10
20001  -  10
98101  -  10
60601  -  10
77002  -  10
30303  -  10
02108  -  7
33131  -  10
80202  -  10
92101  -  10
85004  -  10
98104  -  10
75201  -  10
60611  -  10
75205  -  10
19104  -  10
30363  -  8
98109  -  10


In [24]:
pd.DataFrame(mojeek)

,Title,RankNumber,ZipCode,searchTerm
0,Precita Eyes Muralists Association | San Franc...,1,94110,le specs in 94110
1,Commercial & Industrial Architects in California,2,94110,le specs in 94110
2,Events in the past - MATIZ FLAMENCO,3,94110,le specs in 94110
3,04211-38014 Genuine Toyota CARBURETOR Kit,4,94110,le specs in 94110
4,Vintage | Uncrate Supply,5,94110,le specs in 94110
...,...,...,...,...
190,"Amazon Basics HDMI to DVI Adapter Cable, Black...",6,98109,le specs in 98109
191,Vancouver Shipyards 60 Patrol Boat Yacht Conve...,7,98109,le specs in 98109
192,Racecanam.com Site,8,98109,le specs in 98109
193,43' 2022 Tiara - ALEXANDER MARINE USA™,9,98109,le specs in 98109


## Yahoo Search

In [25]:
yahoo = []

#Get Starting Webpage
url = "https://search.yahoo.com/search;_ylt=AwrEo_2emiRknKkNCTxDDWVH;_ylc=X1MDMTE5NzgwNDg2NwRfcgMyBGZyAwRmcjIDcDpzLHY6c2ZwLG06c2ItdG9wBGdwcmlkA0Y5NkN5Z200U1YyYk5Ia1U3d1BmMkEEbl9yc2x0AzAEbl9zdWdnAzIEb3JpZ2luA3NlYXJjaC55YWhvby5jb20EcG9zAzAEcHFzdHIDBHBxc3RybAMwBHFzdHJsAzE4BHF1ZXJ5A3VsbGElMjBqb2huc29uJTIwMTkxMDQEdF9zdG1wAzE2ODAxMjA0ODQ-?p=ulla+johnson+19104&fr=sfp&fr2=p%3As%2Cv%3Asfp%2Cm%3Asb-top&iscqry="
browser.get(url)

for zipcode in zipcodesSample:
    browser.get(url)
    #Submit & Search
    submit=browser.find_element(By.ID, 'sbq-submit')
    search_form = browser.find_element(By.ID, 'yschsp')
    search_form.clear()
    search_form.send_keys(search_term+zipcode)
    submit.click()
    time.sleep(2)
    
    #Collect Results
    results_box = browser.find_element(By.CLASS_NAME, 'searchCenterMiddle')
    results = results_box.find_elements(By.CLASS_NAME, 'algo')

    #Scraping Through the Results
    '''
    for page in range(pages):
        if page == 1:
            #Load/Click Load More Results
            more_results_button = browser.find_element(By.CLASS_NAME, 'next')
            more_results_button.click()
        time.sleep(2)
    ''' 
    for idx, store in enumerate(results, start=1):
        resultsInfo = {}
        resultsInfo['Title'] = store.find_element(By.CLASS_NAME, 'd-ib').get_attribute("aria-label")
        resultsInfo['RankNumber'] = idx
        #resultsInfo['PageNumber'] = page+1
        resultsInfo['ZipCode'] = zipcode
        resultsInfo['searchTerm'] = search_term+zipcode
        yahoo.append(resultsInfo)
        
    print(zipcode, ' - ', len(results))

94110  -  12
90210  -  13
10001  -  13
20001  -  13
98101  -  12
60601  -  12
77002  -  12
30303  -  12
02108  -  13
33131  -  13
80202  -  13
92101  -  12
85004  -  12
98104  -  13
75201  -  13
60611  -  12
75205  -  12
19104  -  12
30363  -  12
98109  -  13


In [26]:
pd.DataFrame(yahoo).sample(5)#.to_csv("pbrDoubles.csv")

,Title,RankNumber,ZipCode,searchTerm
74,Technical data - B. Braun,12,60601,le specs in 60601
101,Shop Le Specs Online | Nordstrom,3,02108,le specs in 02108
35,Sartorius LE10001 Top-loading Balance | Discon...,11,10001,le specs in 10001
6,Le Specs | Selfridges,7,94110,le specs in 94110
70,NVIDIA GeForce 6600 LE Specs | TechPowerUp GPU...,8,60601,le specs in 60601


## Bing

In [27]:
bing = []

#Get Starting Webpage
url = "https://www.bing.com/search?q=ulla+johnson+19104&form=QBLH&sp=-1&ghc=1&lq=0&pq=ulla+johnson+19104&sc=5-18&qs=n&sk=&cvid=6BC0EAD2980848CEB01D6C706FD6D7AF&ghsh=0&ghacc=0&ghpl="
browser.get(url)

for zipcode in zipcodesSample:
    browser.get(url)
    #Submit & Search
    submit=browser.find_element(By.ID, 'sb_go_par')
    search_form = browser.find_element(By.ID, 'sb_form_q')
    search_form.clear()
    search_form.send_keys(search_term+zipcode)
    submit.click()
    time.sleep(2)
    
    #PAGE 1
    results_box = browser.find_element(By.ID, 'b_results')
    results = results_box.find_elements(By.CLASS_NAME, 'b_algo')
    
    #Scraping Through the Results
    for idx, store in enumerate(results, start=1):
        resultsInfo = {}
        resultsInfo['Title'] = store.find_element(By.TAG_NAME, 'a').get_attribute("textContent")
        resultsInfo['RankNumber'] = idx
        resultsInfo['ZipCode'] = zipcode
        resultsInfo['searchTerm'] = search_term+zipcode
        bing.append(resultsInfo)
    print(zipcode, ' - ', len(results))
    

94110  -  8
90210  -  4
10001  -  4
20001  -  4
98101  -  4
60601  -  7
77002  -  4
30303  -  4
02108  -  4
33131  -  4
80202  -  4
92101  -  4
85004  -  4
98104  -  4
75201  -  4
60611  -  4
75205  -  4
19104  -  4
30363  -  4
98109  -  4


In [28]:
pd.DataFrame(bing)#.to_csv("pbrDoubles.csv")

,Title,RankNumber,ZipCode,searchTerm
0,Store Locator – Le Specs,1,94110,le specs in 94110
1,Le Specs | Designer | NET-A-PORTER,2,94110,le specs in 94110
2,Le Specs,3,94110,le specs in 94110
3,94010 + 94110 — Tudor Sub,4,94110,le specs in 94110
4,94010 + 94110 — Tudor Sub,5,94110,le specs in 94110
...,...,...,...,...
82,Shop Le Specs Online | Nordstrom,4,30363,le specs in 30363
83,Store Locator – Le Specs,1,98109,le specs in 98109
84,Le Specs | Designer | NET-A-PORTER,2,98109,le specs in 98109
85,Shop Le Specs Online | Nordstrom,3,98109,le specs in 98109


## Google

In [29]:
google = []

url = 'https://www.google.com/search?q=google'
browser.get(url)

for zipcode in zipcodesSample:
    base_url = 'https://www.google.com/search?q=' + search_term_raw.replace(" ", "+") + "+in+"+str(zipcode)
    browser.get(base_url)
    
    for i in range(0,2):
        browser.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(3)
        #print('scrolling...')
        try:
            more = browser.find_element(By.CLASS_NAME, 'RVQdVd')
            more.click()
            #print('load more click!')
        except:
            pass
            #print('pass', current_combo, ' - ', i)
    
    print("~done scrolling~")
    results = browser.find_elements(By.CLASS_NAME, 'yuRUbf')
    print(search_term_raw + ' - ', str(len(results)))
    
    for idx, blueLink in enumerate(results, 1):
        resultInfo = {}
        resultInfo['Title'] = blueLink.find_element(By.TAG_NAME, 'a').find_element(By.CLASS_NAME, 'LC20lb').text
        resultInfo['RankNumber'] = idx
        resultInfo['ZipCode'] = zipcode
        resultsInfo['base_url'] = base_url
        resultInfo['link'] = blueLink.find_element(By.TAG_NAME, 'a').get_attribute('href')
        try:
            resultInfo['link_website'] = blueLink.find_element(By.TAG_NAME, 'a').find_element(By.CLASS_NAME, 'VuuXrf').text
        except:
            pass
        google.append(resultInfo)

    browser.quit()
    browser = webdriver.Chrome(options=chrome_options)

    time.sleep(2)

~done scrolling~
le specs -  28
~done scrolling~
le specs -  31
~done scrolling~
le specs -  33
~done scrolling~
le specs -  32
~done scrolling~
le specs -  22
~done scrolling~
le specs -  22
~done scrolling~
le specs -  19
~done scrolling~
le specs -  21
~done scrolling~
le specs -  12
~done scrolling~
le specs -  17
~done scrolling~
le specs -  18
~done scrolling~
le specs -  18
~done scrolling~
le specs -  23
~done scrolling~
le specs -  19
~done scrolling~
le specs -  42
~done scrolling~
le specs -  28
~done scrolling~
le specs -  10
~done scrolling~
le specs -  24
~done scrolling~
le specs -  20
~done scrolling~
le specs -  19


In [30]:
pd.DataFrame(google)

,Title,RankNumber,ZipCode,link,link_website
0,Stockists,1,94110,https://lespecs.com/pages/stockists,Le Specs
1,Le Specs,2,94110,https://lespecs.com/,NaN
2,Shops with Le Specs in San Francisco,3,94110,https://www.thelabelfinder.com/san-francisco/l...,TheLabelFinder
3,Specs' Twelve Adler Museum Cafe,4,94110,https://www.specsbarsf.com/,Specs' Twelve Adler Museum Cafe
4,Self Edge is Denim,5,94110,https://www.selfedge.com/,Self Edge
...,...,...,...,...,...
453,Aaa fire protection - 2023,15,98109,https://cypko.com/777724-aaa-fire-protection,cypko.com
454,The 13 Best Shows to Watch on Discovery Plus R...,16,98109,https://kossfund.online/98109-the-13-best-show...,kossfund.online
455,hobie cat dealers near me,17,98109,https://d-rally.cfd/hobie-cat-dealers-near-me....,d-rally.cfd
456,2020 lagoon 46 - 2023,18,98109,https://didnt.sbs/1213424-2020-lagoon-46,didnt.sbs


## Google Shopping

In [31]:
def get_storeInfo(store, zipcode_here, current_rank_here):
    data = {}
    data['Title'] = store.find_element(By.CLASS_NAME, 'MxVeme').text
    data['RankNumber'] = current_rank_here
    data['ZipCode'] = zipcode_here
    data['google_maps_link'] = store.find_element(By.CLASS_NAME, 'k7eIUb').find_element(By.TAG_NAME, 'a').get_attribute('data-url')
    data['address'] = store.find_element(By.CLASS_NAME, 'lSS0Af').text
    return data

In [35]:
google_shopping = []

url_base = 'https://www.google.com/search?q=*&tbm=shop'
browser.get(url)

for zipcode in zipcodesSample:
    try:
        current_combo = url_base.replace("*", search_term_raw.replace(" ", "+") + "+in+"+str(zipcode))
        print(current_combo)
        browser.get(current_combo) #Get the link
        morePlaces = True

        results = browser.find_element(By.XPATH, '//div[@jscontroller="lcX38e"]')
        stores = results.find_elements(By.CLASS_NAME, 'FFnM0')
        print("LEN:", len(stores))

        while morePlaces == True:
            #for length in range(len(stores)-3): #How many times to click the more places button
            try:
                button = results.find_element(By.CLASS_NAME, 't6JUTe')
                button.click()
                time.sleep(1)
            except:
                pass
                #print("no more 'more places' button")
                morePlaces = False

        for idx, store in enumerate(stores):
            google_shopping.append(get_storeInfo(store, zipcode, idx))

        #time.sleep(1)
    
    except Exception as e:
        print('ERROR', str(e)[0:75]+"...")
    
    browser.quit()
    browser = webdriver.Chrome(options=chrome_options)

https://www.google.com/search?q=le+specs+in+94110&tbm=shop
LEN: 33
no more 'more places' button
https://www.google.com/search?q=le+specs+in+90210&tbm=shop
ERROR Message: no such element: Unable to locate element: {"method":"xpath","sele...
https://www.google.com/search?q=le+specs+in+10001&tbm=shop
LEN: 35
no more 'more places' button
https://www.google.com/search?q=le+specs+in+20001&tbm=shop
ERROR Message: no such element: Unable to locate element: {"method":"xpath","sele...
https://www.google.com/search?q=le+specs+in+98101&tbm=shop
LEN: 25
no more 'more places' button
https://www.google.com/search?q=le+specs+in+60601&tbm=shop
LEN: 37
no more 'more places' button
https://www.google.com/search?q=le+specs+in+77002&tbm=shop
LEN: 67
no more 'more places' button
https://www.google.com/search?q=le+specs+in+30303&tbm=shop
LEN: 39
no more 'more places' button
https://www.google.com/search?q=le+specs+in+02108&tbm=shop
LEN: 28
no more 'more places' button
https://www.google.com/search?q=le+specs

In [36]:
pd.DataFrame(google_shopping)

,Title,RankNumber,ZipCode,google_maps_link,address
0,Nordstrom,0,94110,https://maps.google.com/maps?daddr=1870+Redwoo...,"1870 Redwood Hwy, Corte Madera·13.7 miles"
1,Neiman Marcus,1,94110,https://maps.google.com/maps?daddr=150+Stockto...,"150 Stockton Street, San Francisco·2.8 miles"
2,Sunglass Hut,2,94110,https://maps.google.com/maps?daddr=2485+Sand+C...,"2485 Sand Creek Rd, #108, Spc# G-7, Brentwood"
3,Nordstrom,3,94110,https://maps.google.com/maps?daddr=865+Market+...,"865 Market St, San Francisco·2.5 miles"
4,Nordstrom,4,94110,https://maps.google.com/maps?daddr=1200+Broadw...,"1200 Broadway Plaza, Walnut Creek·22.0 miles"
...,...,...,...,...,...
581,Kohl's,14,98109,https://maps.google.com/maps?daddr=2909+Bickfo...,"2909 Bickford Ave, Snohomish·24.1 miles"
582,Macy's,15,98109,https://maps.google.com/maps?daddr=400+Bellevu...,"400 Bellevue Square, Bellevue·6.6 miles"
583,Sportsman's Warehouse,16,98109,https://maps.google.com/maps?daddr=505+SE+Ever...,"505 SE Everett Mall Way, Ste 1, Everett·19.8 m..."
584,West Marine,17,98109,https://maps.google.com/maps?daddr=1400+NW+45t...,"1400 NW 45th St, Seattle·2.5 miles"


## Official

In [3]:
resultsList = []
url = brand_url

browser.get(url)
time.sleep(2)

for i, zipcode in enumerate(zipcodesSample, start=1):
    browser.get(url)
    
    query_entry=browser.find_element(By.CLASS_NAME, 'stockist-query-entry')
    input_field = query_entry.find_element(By.TAG_NAME, 'input')
    submit = query_entry.find_element(By.CLASS_NAME, 'stockist-search-button').find_element(By.TAG_NAME, 'button')

    input_field.clear()
    input_field.send_keys(zipcode)
    submit.click()
    
    time.sleep(2)
    search_results = browser.find_element(By.CLASS_NAME, 'stockist-result-list')
    res=search_results.find_elements(By.CLASS_NAME,'stockist-result')
    
    for idx, store in enumerate(res):
        storeInfo = {}
        storeInfo['store_name'] = store.find_element(By.CLASS_NAME, 'stockist-result-name').get_attribute("textContent")
        storeInfo['RankNumber'] = idx
        storeInfo['ZipCode'] = zipcode
        address = [line.get_attribute("textContent") for line in 
                  store.find_element(By.CLASS_NAME, 'stockist-result-address').find_elements(By.TAG_NAME, 'div')]
        storeInfo['address'] = ", ".join(address)
        storeInfo['distance'] = store.find_element(By.CLASS_NAME, 'stockist-result-distance-text').get_attribute("textContent")
        storeInfo['google_maps'] = store.find_element(By.CLASS_NAME, 'stockist-result-directions-link').find_element(By.TAG_NAME, 'a').get_attribute('href')
        storeInfo['starting_url(brand)'] = url
        #If website link in results
        '''
        try:
            storeInfo['website_link'] = store.find_element(By.CLASS_NAME, 'stockist-result-website').find_element(By.TAG_NAME, 'a').get_attribute('href')
            print(storeInfo['website_link'])
        except:
            pass
            #print('...no weblink', storeInfo['name'], zipcode)
        '''
        resultsList.append(storeInfo)
    print(zipcode, " results:", len(res), ' -', i)
    
    #except Exception as e:
    #    print("ERROR", zipcode, i, str(e)[0:75]+"...")
    #    time.sleep(2)
print("===Done===")

94110  results: 9  - 1
90210  results: 3  - 2
10001  results: 3  - 3
20001  results: 3  - 4
98101  results: 3  - 5
60601  results: 3  - 6
77002  results: 3  - 7
30303  results: 0  - 8
02108  results: 0  - 9
33131  results: 0  - 10
80202  results: 0  - 11
92101  results: 0  - 12
85004  results: 0  - 13
98104  results: 0  - 14
75201  results: 8  - 15
60611  results: 3  - 16
75205  results: 3  - 17


KeyboardInterrupt: 

In [ ]:
pd.DataFrame(resultsList)

# Meta Comparison & Fuzzy Wuzzy

### Convert all to df

In [75]:
bing_df = pd.DataFrame(bing)
yahoo_df = pd.DataFrame(yahoo)
mojeek_df = pd.DataFrame(mojeek)
startPage_df = pd.DataFrame(startPage)
duckduckGo_df = pd.DataFrame(duckduckGo)
braveSearch_df = pd.DataFrame(braveSearch)
yellowPages_df = pd.DataFrame(yellowPages)
google_df = pd.DataFrame(google)
results_df = pd.DataFrame(resultsList)
google_shopping_df = pd.DataFrame(google_shopping)

df_list = [duckduckGo_df, startPage_df, braveSearch_df, yellowPages_df,
           mojeek_df, yahoo_df, bing_df, google_df, google_shopping_df] 
           #results_df]

df_list_names = ['duckduckGo_df', 'startPage_df', 'braveSearch_df', 'yellowPages_df', 
                 'mojeek_df', 'yahoo_df', 'bing_df', 'google_df', 
                 'google_shopping_df'] #'results_df']


In [76]:
for df in df_list:
    print(f"Df length: {len(df)}, > {df.isnull().any(axis=1).sum()}")
    for column in df.columns:
        print(column, end="\t")
    print('\n')
    if df.isnull().any(axis=1).sum() > 0:
        nan_rows = df[df.isnull().any(axis=1)]
        print(f"The following rows in the DataFrame have NaN values:\n{nan_rows}\n")

average_length = sum(len(df) for df in df_list) / len(df_list)
print('\n')
print('\nAverage length', average_length)

Df length: 200, > 0
Title	RankNumber	ZipCode	searchTerm	

Df length: 178, > 0
Title	RankNumber	ZipCode	searchTerm	link	

Df length: 18, > 0
Title	RankNumber	ZipCode	searchTerm	

Df length: 598, > 0
Title	RankNumber	ZipCode	searchTerm	

Df length: 195, > 0
Title	RankNumber	ZipCode	searchTerm	

Df length: 249, > 0
Title	RankNumber	ZipCode	searchTerm	

Df length: 87, > 86
Title	RankNumber	ZipCode	searchTerm	base_url	

The following rows in the DataFrame have NaN values:
                                 Title  RankNumber ZipCode         searchTerm  \
0             Store Locator – Le Specs           1   94110  le specs in 94110   
1   Le Specs | Designer | NET-A-PORTER           2   94110  le specs in 94110   
2                             Le Specs           3   94110  le specs in 94110   
3            94010 + 94110 — Tudor Sub           4   94110  le specs in 94110   
4            94010 + 94110 — Tudor Sub           5   94110  le specs in 94110   
..                                 ...    

### Fuzzy Wuzzy Algorithm

In [77]:
s1 = "Le Specs | Designer | NET-A-PORTER"
s2 = "Le Specs Stockist"
s3 = "Airstream For Sale In Texas - 2023"

sample = 'Urban Outfitters'
sample1 = 'dress San Francisco'
print("FuzzyWuzzy Ratio: ", fuzz.ratio(sample, sample1))
print("FuzzyWuzzy PartialRatio: ", fuzz.partial_ratio(sample, sample1))
print("FuzzyWuzzy TokenSortRatio: ", fuzz.token_sort_ratio(sample, sample1))
print("FuzzyWuzzy TokenSetRatio: ", fuzz.token_set_ratio(sample, sample1))
print("FuzzyWuzzy WRatio: ", fuzz.WRatio(sample, sample1))

FuzzyWuzzy Ratio:  34
FuzzyWuzzy PartialRatio:  33
FuzzyWuzzy TokenSortRatio:  34
FuzzyWuzzy TokenSetRatio:  34
FuzzyWuzzy WRatio:  40


In [78]:
#### Metric
#Average of WRatio & TokenSetRaio

def WTokenRatio(str1, str2):
    wratio = fuzz.WRatio(str1, str2)
    token_set = fuzz.token_set_ratio(str1, str2)
    return (wratio+token_set)/2

def extractTopWTokenRatio(search_result, choices):
    val_list = []
    
    for choice in choices:
        val_list.append((choice, WTokenRatio(search_result, choice)))
    
    max_ratio_item = max(val_list, key=lambda x: x[1])

    return max_ratio_item

In [79]:
print(WTokenRatio(s1, s2))
print(WTokenRatio(s1, s3))

75.0
34.0


### Testing Custom Function

In [80]:
stores_with_brand

['Seattle Eye',
 'Wayward - Seattle',
 'Verdis',
 'Evereve',
 'Lika Love',
 'Evereve',
 'Evereve',
 'Wayward',
 'Evereve']

In [81]:
search_result_item_1 = 'Le Specs Sunglasses for Women | Nordstrom'
search_result_item_2 = 'Le Specs Store Locator'

#Turn results into a list
stores_with_brand = results_df.query("ZipCode=='98101'")["store_name"].tolist()
print(" - ".join(stores_with_brand), '\n')

print(search_result_item_1)
for choice in stores_with_brand:
    print(choice.ljust(20), WTokenRatio(search_result_item_1, choice))
print("TOP:", extractTopWTokenRatio(search_result_item_1, stores_with_brand))

print('\n')

print(search_result_item_2)
for choice in stores_with_brand:
    print(choice.ljust(20), WTokenRatio(search_result_item_2, choice))
print("TOP:", extractTopWTokenRatio(search_result_item_2, stores_with_brand))

Seattle Eye - Wayward - Seattle - Verdis - Evereve - Lika Love - Evereve - Evereve - Wayward - Evereve 

Le Specs Sunglasses for Women | Nordstrom
Seattle Eye          27.5
Wayward - Seattle    27.0
Verdis               31.5
Evereve              21.5
Lika Love            25.5
Evereve              21.5
Evereve              21.5
Wayward              17.5
Evereve              21.5
TOP: ('Verdis', 31.5)


Le Specs Store Locator
Seattle Eye          37.0
Wayward - Seattle    31.5
Verdis               25.5
Evereve              28.0
Lika Love            35.0
Evereve              28.0
Evereve              28.0
Wayward              21.0
Evereve              28.0
TOP: ('Seattle Eye', 37.0)


### One Engine Audit

In [82]:
#Correct defined as having a 70 or higher 'WTokenRatio' number
threshold = 70
total_correct = 0
total_incorrect = 0
correct_dict = []
incorrect_dict = []

for zipcode in zipcodesSample: #Zipcodes
    #Store Locator
    store_locations_df = results_df.query(f"ZipCode=='{zipcode}'") #DataFrame
    store_locations = store_locations_df["store_name"].tolist()
    store_locations.append("Stockist Store Locator")

    #Search Engine
    search_results_df = duckduckGo_df.query(f"ZipCode == '{zipcode}'") #####
    search_results = search_results_df['Title'].tolist()

    print(zipcode)
    for idx, result in enumerate(search_results):
        if len(store_locations) > 0:
            extraction = extractTopWTokenRatio(result, store_locations)
            print(idx, result.ljust(70), extraction, '\t')
            
            if extraction[1] >= threshold:
                total_correct+=1
                entry = search_results_df.iloc[idx].to_dict()
                entry['Search Engine'] = 'duckduckgo' #####
                correct_dict.append(entry)
            else:
                total_incorrect+=1
                entry = search_results_df.iloc[idx].to_dict()
                entry['Search Engine'] = 'duckduckgo' #####
                incorrect_dict.append(entry)
        else:
            total_incorrect+=1
    print('\n')

print(total_correct, total_incorrect, total_correct/(total_correct+total_incorrect))

94110
0 Store Locator - Le Specs                                               ('Stockist Store Locator', 75.0) 	
1 Le Specs                                                               ('Stockist Store Locator', 38.0) 	
2 Le Specs | Designer | NET-A-PORTER                                     ('Turner And Co', 43.0) 	
3 Shop Le Specs Online | Nordstrom                                       ('Stockist Store Locator', 41.5) 	
4 Shop Le Specs Online | Shopbop                                         ('Stockist Store Locator', 37.0) 	
5 Le Specs Sunglasses Review - Must Read This Before Buying              ('Turner And Co', 32.5) 	
6 Home - Le Specs                                                        ('Stockist Store Locator', 28.5) 	
7 Our Story - Le Specs                                                   ('Stockist Store Locator', 58.5) 	
8 Le Specs | Selfridges                                                  ('Stockist Store Locator', 38.0) 	
9 94010 + 94110 — Tudor Sub             

### All Engines

In [83]:
df_list_names

['duckduckGo_df',
 'startPage_df',
 'braveSearch_df',
 'yellowPages_df',
 'mojeek_df',
 'yahoo_df',
 'bing_df',
 'google_df',
 'google_shopping_df']

In [84]:
correct_dict, incorrect_dict, threshold = [], [], 70

for df_num, df in enumerate(df_list):
    print(df_num, df_list_names[df_num], " results:")
    for zipcode in zipcodesSample:
        print(zipcode)
        
        store_locations_df = results_df.query(f"ZipCode=='{zipcode}'") #DataFrame
        store_locations = store_locations_df["store_name"].tolist()
        store_locations.extend(["Stockist Store Locator", "Stockist", "Stockists"])

        #Search Engine
        search_results_df = df.query(f"ZipCode == '{zipcode}'") #####
        search_results = search_results_df['Title'].tolist()
        
        for idx, result in enumerate(search_results):
            if len(store_locations) > 0 and len(search_results) > 0:
                extraction = extractTopWTokenRatio(result, store_locations)
                #print(idx, result.ljust(70), extraction, '\t')

                if extraction[1] >= threshold:
                    entry = search_results_df.iloc[idx].to_dict()
                    entry['Search Engine'] = df_list_names[df_num] #####
                    entry['Extraction Similarity Name'] = extraction[0]
                    entry['Extraction Similarity Score'] = extraction[1]
                    entry['Met Threshold?'] = 1
                    #print(extraction)
                    correct_dict.append(entry)
                else:
                    entry = search_results_df.iloc[idx].to_dict()
                    entry['Search Engine'] = df_list_names[df_num] #####
                    entry['Extraction Similarity Name'] = extraction[0]
                    entry['Extraction Similarity Score'] = extraction[1]
                    entry['Met Threshold?'] = 0
                    incorrect_dict.append(entry)
            else:
                print(df_num, "Empty")

0 duckduckGo_df  results:
94110
90210
10001
20001
98101
60601
77002
30303
02108
33131
80202
92101
85004
98104
75201
60611
75205
19104
30363
98109
1 startPage_df  results:
94110
90210
10001
20001
98101
60601
77002
30303
02108
33131
80202
92101
85004
98104
75201
60611
75205
19104
30363
98109
2 braveSearch_df  results:
94110
90210
10001
20001
98101
60601
77002
30303
02108
33131
80202
92101
85004
98104
75201
60611
75205
19104
30363
98109
3 yellowPages_df  results:
94110
90210
10001
20001
98101
60601
77002
30303
02108
33131
80202
92101
85004
98104
75201
60611
75205
19104
30363
98109
4 mojeek_df  results:
94110
90210
10001
20001
98101
60601
77002
30303
02108
33131
80202
92101
85004
98104
75201
60611
75205
19104
30363
98109
5 yahoo_df  results:
94110
90210
10001
20001
98101
60601
77002
30303
02108
33131
80202
92101
85004
98104
75201
60611
75205
19104
30363
98109
6 bing_df  results:
94110
90210
10001
20001
98101
60601
77002
30303
02108
33131
80202
92101
85004
98104
75201
60611
75205
19104
3036

### Aim → From the top X Google results, how many are on the store list? Use names/addresses
EX:
* 17% on Google
* 5% on Google Shopping
* 22% on DuckDuckGo
* 8% on Bing



In [87]:
len(pd.DataFrame(correct_dict))/  (len(pd.DataFrame(incorrect_dict)) + len(pd.DataFrame(correct_dict)) )

0.04671078240560529

In [88]:
combined_df = pd.concat([pd.DataFrame(correct_dict), pd.DataFrame(incorrect_dict)])
combined_df.to_csv("lespecs.csv")

In [90]:
combined_df

,Title,RankNumber,ZipCode,searchTerm,Search Engine,Extraction Similarity Name,Extraction Similarity Score,Met Threshold?,link,base_url,link_website,google_maps_link,address
0,Store Locator - Le Specs,1,94110,le specs in 94110,duckduckGo_df,Stockist Store Locator,75.0,1,NaN,NaN,NaN,NaN,NaN
1,Store Locator - Le Specs,1,90210,le specs in 90210,duckduckGo_df,Stockist Store Locator,75.0,1,NaN,NaN,NaN,NaN,NaN
2,Store Locator - Le Specs,1,20001,le specs in 20001,duckduckGo_df,Stockist Store Locator,75.0,1,NaN,NaN,NaN,NaN,NaN
3,Store Locator - Le Specs,1,98101,le specs in 98101,duckduckGo_df,Stockist Store Locator,75.0,1,NaN,NaN,NaN,NaN,NaN
4,Store Locator - Le Specs,1,60601,le specs in 60601,duckduckGo_df,Stockist Store Locator,75.0,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2444,Kohl's,14,98109,NaN,google_shopping_df,Stockist Store Locator,33.0,0,NaN,NaN,NaN,https://maps.google.com/maps?daddr=2909+Bickfo...,"2909 Bickford Ave, Snohomish·24.1 miles"
2445,Macy's,15,98109,NaN,google_shopping_df,Turner And Co,31.0,0,NaN,NaN,NaN,https://maps.google.com/maps?daddr=400+Bellevu...,"400 Bellevue Square, Bellevue·6.6 miles"
2446,Sportsman's Warehouse,16,98109,NaN,google_shopping_df,Turner And Co,37.0,0,NaN,NaN,NaN,https://maps.google.com/maps?daddr=505+SE+Ever...,"505 SE Everett Mall Way, Ste 1, Everett·19.8 m..."
2447,West Marine,17,98109,NaN,google_shopping_df,Stockist Store Locator,38.0,0,NaN,NaN,NaN,https://maps.google.com/maps?daddr=1400+NW+45t...,"1400 NW 45th St, Seattle·2.5 miles"
